<a href="https://colab.research.google.com/github/RedietNegash/Machine-Learning/blob/main/content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt

In [ ]:
df_articles = pd.read_csv("/content/drive/MyDrive/Machine-Learning/Recommendation-Systems/articles_metadata.csv", low_memory=False)


In [ ]:
print(df_articles.head())

   articleId                           title  \
0          1     The Impact of AI on Society   
1          2  Understanding Machine Learning   
2          3         Deep Learning Explained   
3          4          The Future of Robotics   
4          5                       AI Ethics   

                                             content  
0              AI is transforming various sectors...  
1              Machine learning is a subset of AI...  
2  Deep learning is a technique in machine learni...  
3  Robots are increasingly becoming part of our d...  
4  Ethical considerations in AI development are c...  


In [ ]:
df_ratings=pd.read_csv("/content/drive/MyDrive/Machine-Learning/Recommendation-Systems/article_ratings_small.csv")
df_ratings.head(100)

,userId,articleId,rating,timestamp
0,1,1,4.0,1622555555
1,1,2,5.0,1622555556
2,1,3,4.0,1622555557
3,1,4,3.0,1622555558
4,1,5,5.0,1622555559
5,2,1,3.0,1622555560
6,2,3,4.0,1622555561
7,2,6,2.0,1622555562
8,2,7,5.0,1622555563
9,2,10,4.0,1622555564


In [ ]:
# df_links=pd.read_csv("/content/drive/MyDrive/Machine-Learning/Recommendation-Systems/links_small.csv")
# df_links.head(10)

In [ ]:
# movie_862=df_links[df_links['movieId']==1]
# movie_862

,movieId,imdbId,tmdbId
0,1,114709,862.0


In [ ]:
df_merged = pd.merge(df_ratings, df_articles, on="articleId", how="left")
print(df_merged.head())

   userId  articleId  rating   timestamp                           title  \
0       1          1     4.0  1622555555     The Impact of AI on Society   
1       1          2     5.0  1622555556  Understanding Machine Learning   
2       1          3     4.0  1622555557         Deep Learning Explained   
3       1          4     3.0  1622555558          The Future of Robotics   
4       1          5     5.0  1622555559                       AI Ethics   

                                             content  \
0              AI is transforming various sectors...   
1              Machine learning is a subset of AI...   
2  Deep learning is a technique in machine learni...   
3  Robots are increasingly becoming part of our d...   
4  Ethical considerations in AI development are c...   

                                           embedding  
0  [0.0065397914, -0.02900096, -0.007339295, -0.0...  
1  [-0.015036831, -0.03257998, 0.07369401, 0.0105...  
2  [-0.036377545, -0.03372445, 0.01336774

In [ ]:
def calculate_similarity(articles):
    similarities = []
    for i in range(len(articles)):
        for j in range(i + 1, len(articles)):
            article1 = articles.iloc[i]
            article2 = articles.iloc[j]

            # Calculate content similarity (using a simple overlap of keywords for demonstration)
            content1 = set(article1['content'].split())
            content2 = set(article2['content'].split())
            shared_content = len(content1.intersection(content2))
            total_content = len(content1.union(content2))
            content_similarity = shared_content / total_content if total_content > 0 else 0

            # Calculate rating similarity
            rating1 = article1['rating']
            rating2 = article2['rating']
            rating_similarity = 1 - abs(rating1 - rating2) / max(rating1, rating2) if rating1 and rating2 else 0

            # Combine similarities: weighted average
            overall_similarity = (content_similarity + rating_similarity) / 2

            # Append results
            similarities.append({
                'articleId': article1['articleId'],
                'sim_articleId': article2['articleId'],
                'relevance': overall_similarity
            })

    return similarities

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
df_articles['content'] = df_articles['content'].fillna('')
all_content = df_articles['content'].tolist()
all_embeddings = model.encode(all_content)

df_articles = df_articles.copy()
df_articles['embedding'] = list(all_embeddings)


print(df_articles[['articleId', 'content', 'embedding']].head())

   articleId                                            content  \
0          1              AI is transforming various sectors...   
1          2              Machine learning is a subset of AI...   
2          3  Deep learning is a technique in machine learni...   
3          4  Robots are increasingly becoming part of our d...   
4          5  Ethical considerations in AI development are c...   

                                           embedding  
0  [0.0065397914, -0.02900096, -0.007339295, -0.0...  
1  [-0.015036831, -0.03257998, 0.07369401, 0.0105...  
2  [-0.036377545, -0.03372445, 0.013367745, -0.01...  
3  [-0.0032222318, -0.05688273, 0.08174227, -0.00...  
4  [-0.026047785, 0.041954458, -0.0559622, -0.046...  


In [ ]:
all_embeddings.shape

(15, 384)

In [ ]:
def compute_weighted_average(df_ratings):
    embeddings = np.array(df_articles['embedding'].tolist())
    ratings = df_ratings['rating'].values
    weighted_avg_embedding = np.average(embeddings, axis=0, weights=ratings)
    return weighted_avg_embedding

In [ ]:
user_profiles = df_merged.groupby('userId').apply(compute_weighted_average).reset_index()
user_profiles.columns = ['userId', 'profile_embedding']
user_profiles.head()


<ipython-input-27-ab31c9b0c58c>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_profiles = df_merged.groupby('userId').apply(compute_weighted_average).reset_index()


,userId,profile_embedding
0,1,"[-0.015925751865974495, -0.01784178330784752, ..."
1,2,"[-0.002341887059931954, -0.012313246313068602,..."
2,3,"[0.007804625471563716, -0.009231027926465398, ..."
3,4,"[-0.023701064055785537, -0.0268607372418046, 0..."
4,5,"[0.0015445607196953562, -0.03116066836648517, ..."


In [ ]:
def get_relevant_articles(user_id, threshold=3.5):
    relevant_articles = df_merged[(df_merged['userId'] == user_id) & (df_merged['rating'] >= threshold)]['articleId']
    return set(relevant_articles)

In [ ]:
def compute_similarity(profile_embedding, movie_embeddings):
    return cosine_similarity([profile_embedding], all_embeddings )

In [ ]:
def generate_recommendations_for_user_con(user_id, k=10):
    if user_id in user_profiles['userId'].values:
        profile_embedding = user_profiles[user_profiles['userId'] == user_id]['profile_embedding'].values[0]


        all_article_embeddings = np.array(df_articles['embedding'].tolist())


        similarities = compute_similarity(profile_embedding, all_article_embeddings)[0]


        all_articles_with_scores = df_articles[['content', 'articleId']].copy()
        all_articles_with_scores['similarity'] = similarities


        top_recommendations = all_articles_with_scores.sort_values(by='similarity', ascending=False).head(k)

        top_recommendations.reset_index(drop=True, inplace=True)

        print(f"Top {k} Recommendations for User ID {user_id}:")
        print(top_recommendations[['content', 'articleId', 'similarity']])

        return top_recommendations
    else:
        print(f"No user profile found for User ID {user_id}.")
        return None

In [ ]:


def extract_unseen_movies(user_id, recommendations):
    seen_movie_ids = get_relevant_articles(user_id)
    unseen_movies = recommendations[~recommendations['articleId'].isin(seen_movie_ids)]
    return unseen_movies

In [ ]:
user_id = 3
all_recommendations = generate_recommendations_for_user_con(user_id, k=10)

Top 10 Recommendations for User ID 3:
                                             content  articleId  similarity
0  Computer vision is a field that enables machin...          8    0.745435
1  Robots are increasingly becoming part of our d...          4    0.709408
2              Machine learning is a subset of AI...          2    0.684573
3              AI is transforming various sectors...          1    0.609886
4  AI applications in healthcare are revolutioniz...         11    0.592066
5  Data science combines statistics, computer sci...          6    0.588825
6  Deep learning is a technique in machine learni...          3    0.581179
7  Ethical considerations in AI development are c...          5    0.547763
8  Reinforcement learning is a type of machine le...         10    0.528841
9  AI can play a significant role in addressing c...         12    0.507852


In [ ]:
def evaluate_recommendations(k=10, relevance_threshold=3.5):
    precision_sum = 0
    recall_sum = 0
    total_rmse = 0
    mrr_sum = 0
    user_count = 0

    for user_id in user_profiles['userId']:
        user_count += 1

        all_recommendations = generate_recommendations_for_user_con(user_id, k=k)
        if all_recommendations is None:
            continue
        relevant_movies = get_relevant_articles(user_id, threshold=relevance_threshold)



        recommended_ids = set(all_recommendations['clean_imdbId'])
        relevant_and_recommended = recommended_ids & relevant_movies


        precision = len(relevant_and_recommended) / k


        recall = len(relevant_and_recommended) / len(relevant_movies) if len(relevant_movies) > 0 else 0

        precision_sum += precision
        recall_sum += recall


        precision_at_k = precision_sum / user_count
        recall_at_k = recall_sum / user_count
        f1_at_k = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0


        print(f"Precision@{k}: {precision_at_k}")
        print(f"Recall@{k}: {recall_at_k}")
        print(f"F1 Score@{k}: {f1_at_k}")
    evaluate_recommendations(k=10)


In [ ]:
def evaluate_recommendations(k=10, relevance_threshold=3.5):
    precision_sum = 0
    recall_sum = 0
    user_count = 0

    for user_id in user_profiles['userId']:
        user_count += 1
        all_recommendations = generate_recommendations_for_user_con(user_id, k=k)
        if all_recommendations is None:
            continue

        relevant_movies = get_relevant_articles(user_id, threshold=relevance_threshold)
        recommended_ids = set(all_recommendations['articleId'])
        relevant_and_recommended = recommended_ids & relevant_movies

        precision = len(relevant_and_recommended) / k
        recall = len(relevant_and_recommended) / len(relevant_movies) if len(relevant_movies) > 0 else 0

        precision_sum += precision
        recall_sum += recall

    precision_at_k = precision_sum / user_count
    recall_at_k = recall_sum / user_count
    f1_at_k = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0

    print(f"Precision@{k}: {precision_at_k}")
    print(f"Recall@{k}: {recall_at_k}")
    print(f"F1 Score@{k}: {f1_at_k}")

# Call the evaluation function
evaluate_recommendations(k=10)

Top 10 Recommendations for User ID 1:
                                             content  articleId  similarity
0              Machine learning is a subset of AI...          2    0.834161
1              AI is transforming various sectors...          1    0.772954
2  Ethical considerations in AI development are c...          5    0.734210
3  Deep learning is a technique in machine learni...          3    0.707602
4  Robots are increasingly becoming part of our d...          4    0.618873
5  Reinforcement learning is a type of machine le...         10    0.593728
6  AI can play a significant role in addressing c...         12    0.589508
7  AI applications in healthcare are revolutioniz...         11    0.565175
8  Computer vision is a field that enables machin...          8    0.540962
9  GANs are a class of machine learning framework...          9    0.515122
Top 10 Recommendations for User ID 2:
                                             content  articleId  similarity
0  Deep lear

-Precision@10 means that, on average, 11.06% of the top 10 recommended movies are relevant to the user (i.e., they match the user's preferences or previous high ratings).


-Recall@K measures the proportion of all relevant items that are recommended within the top K. Recall@10 in this case indicates that the system, on average, retrieves 35.48% of all the relevant movies for the user within the top 10 recommendations.




Precision measures how much of the recommended content is relevant to the user.
Recall measures how much of the relevant content is recommended to the user.

## sparse representation


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np


user_profile_embeddings = np.array(user_profiles['profile_embedding'].tolist())


sparse_user_profiles = csr_matrix(user_profile_embeddings)

print(sparse_user_profiles)


NameError: name 'user_profiles' is not defined

In [ ]:
collabarative
movieId  rating
17      2.50
39      2.50
1172    2.00
47      2.00
50      2.00
10      2.00
1061    1.50
1029    1.50
31      1.25

contentbased
                            clean_imdbId  similarity
0           Now and Then      17          0.199650
1       Cutthroat Island      0112760     0.183249
2           It Takes Two      0113442     0.169755
3         Twelve Monkeys      1029        0.166071
4               Clueless      0112697    0.153906
5             Carrington      0112637    0.153042

hybrid
1, normalize - z score sklean library
merge

## **Hybrid**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Mounted at /content/drive


In [ ]:
df_articles.head()
df_ratings_subset = df_ratings.copy()

df_ratings_subset = df_ratings_subset.groupby('userId').head(5)
df_ratings_subset

,userId,articleId,rating,timestamp
0,1,1,4.0,1622555555
1,1,2,5.0,1622555556
2,1,3,4.0,1622555557
3,1,4,3.0,1622555558
4,1,5,5.0,1622555559
5,2,1,3.0,1622555560
6,2,3,4.0,1622555561
7,2,6,2.0,1622555562
8,2,7,5.0,1622555563
9,2,10,4.0,1622555564


In [ ]:

pivoted_data =df_ratings_subset.pivot(index='userId', columns='articleId', values='rating')
pivoted_data



articleId,1,2,3,4,5,6,7,8,9,10,11
userId,,,,,,,,,,,
1,4.0,5.0,4.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,4.0,NaN,NaN,2.0,5.0,NaN,NaN,4.0,NaN
3,NaN,NaN,NaN,5.0,2.0,4.0,NaN,5.0,NaN,NaN,3.0
4,NaN,4.0,5.0,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
pivoted_data_filled = pivoted_data.fillna(0)


In [ ]:
user_similarity = cosine_similarity(pivoted_data_filled)
user_similarity

array([[1.        , 0.35082321, 0.29485315, 0.51613977, 0.26194334],
       [0.35082321, 1.        , 0.10757898, 0.58848989, 0.41065937],
       [0.29485315, 0.10757898, 1.        , 0.        , 0.35141842],
       [0.51613977, 0.58848989, 0.        , 1.        , 0.        ],
       [0.26194334, 0.41065937, 0.35141842, 0.        , 1.        ]])

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity, index=pivoted_data_filled.index, columns=pivoted_data_filled.index)
user_similarity_df

userId,1,2,3,4,5
userId,,,,,
1,1.000000,0.350823,0.294853,0.51614,0.261943
2,0.350823,1.000000,0.107579,0.58849,0.410659
3,0.294853,0.107579,1.000000,0.00000,0.351418
4,0.516140,0.588490,0.000000,1.00000,0.000000
5,0.261943,0.410659,0.351418,0.00000,1.000000


In [ ]:
def get_k_nearest_neighbors(similarity_matrix, target_user, k):
    similarities = similarity_matrix.loc[target_user]
    similar_users = similarities.sort_values(ascending=False).index[1:k+1]
    print("simliar users")
    print(similar_users)
    return similar_users

In [ ]:
def recommend_movies_coll(ratings, similarity_matrix, target_user, k):

    if target_user not in similarity_matrix.index:
        print(f"Target user {target_user} not found in similarity matrix.")
        return None


    nearest_neighbors = get_k_nearest_neighbors(similarity_matrix, target_user, k)
    neighbor_ratings = ratings.loc[nearest_neighbors]
    avg_ratings = neighbor_ratings.mean()
    user_ratings = ratings.loc[target_user]



    recommendations = avg_ratings
    print("Recommendations including seen articles")
    print(recommendations)

    recommendations = avg_ratings[user_ratings == 0]
    print("Filtered recommendations (excluding articles already rated by target user):")
    print(recommendations)


    recommendations = recommendations.sort_values(ascending=False)

    return recommendations


In [ ]:
k = 4
target_user = 3
recommended_movies = recommend_movies_coll(pivoted_data_filled, user_similarity_df, target_user, k)


simliar users
Index([5, 1, 2, 4], dtype='int64', name='userId')
Recommendations including seen articles
articleId
1     2.75
2     2.25
3     3.25
4     0.75
5     1.25
6     1.75
7     2.25
8     0.00
9     0.75
10    1.00
11    0.00
dtype: float64
Filtered recommendations (excluding articles already rated by target user):
articleId
1     2.75
2     2.25
3     3.25
7     2.25
9     0.75
10    1.00
dtype: float64


In [ ]:
def evaluate_collaborative_filtering(k=10, relevance_threshold=3.5):
    precision_sum = 0
    recall_sum = 0
    user_count = 0

    for user_id in user_similarity_df.index:
        user_count += 1
        recommended_movies = recommend_movies_coll(pivoted_data_filled, user_similarity_df, user_id, k)
        if recommended_movies is None:
            continue


        user_ratings = df_ratings[df_ratings['userId'] == user_id]
        relevant_movies = set(user_ratings[user_ratings['rating'] >= relevance_threshold]['articleId'])


        recommended_ids = set(recommended_movies.index)
        relevant_and_recommended = recommended_ids & relevant_movies

        precision = len(relevant_and_recommended) / k
        recall = len(relevant_and_recommended) / len(relevant_movies) if len(relevant_movies) > 0 else 0

        precision_sum += precision
        recall_sum += recall

    precision_at_k = precision_sum / user_count
    recall_at_k = recall_sum / user_count
    f1_at_k = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0

    return precision_at_k, recall_at_k, f1_at_k
precision_at_k, recall_at_k, f1_at_k = evaluate_collaborative_filtering(k=10)

print(f"Precision@{k}: {precision_at_k}")
print(f"Recall@{k}: {recall_at_k}")
print(f"F1 Score@{k}: {f1_at_k}")


simliar users
Index([4, 2, 3, 5], dtype='int64', name='userId')
Recommendations including seen articles
articleId
1     1.75
2     1.00
3     2.25
4     1.25
5     0.50
6     2.75
7     2.25
8     1.25
9     0.75
10    1.00
11    0.75
dtype: float64
Filtered recommendations (excluding articles already rated by target user):
articleId
6     2.75
7     2.25
8     1.25
9     0.75
10    1.00
11    0.75
dtype: float64
simliar users
Index([4, 5, 1, 3], dtype='int64', name='userId')
Recommendations including seen articles
articleId
1     2.00
2     2.25
3     2.25
4     2.00
5     1.75
6     2.25
7     1.00
8     1.25
9     0.75
10    0.00
11    0.75
dtype: float64
Filtered recommendations (excluding articles already rated by target user):
articleId
2     2.25
4     2.00
5     1.75
8     1.25
9     0.75
11    0.75
dtype: float64
simliar users
Index([5, 1, 2, 4], dtype='int64', name='userId')
Recommendations including seen articles
articleId
1     2.75
2     2.25
3     3.25
4     0.75
5     1.

In [ ]:
def hybrid_recommend_items(cb_recommender, cf_recommender, user_id, cb_weight=1.0, cf_weight=1.0, topn=10):

    cb_recs = cb_recommender(user_id, k=1000)
    cf_recs = cf_recommender(pivoted_data_filled, user_similarity_df, user_id, k=1000)


    if cb_recs is None or cf_recs is None:
        return None


    cb_recs = cb_recs.rename(columns={'similarity': 'recStrengthCB'})
    cf_recs = cf_recs.reset_index().rename(columns={0: 'recStrengthCF'})
    cf_recs.columns = ['articleId', 'recStrengthCF']


    cf_recs['articleId'] = cf_recs['articleId'].astype(str)


    merged_recs = cb_recs.merge(cf_recs, left_on='articleId', right_on='articleId', how='outer').fillna(0)
    print("merged_recs",merged_recs)
    merged_recs['recStrengthHybrid'] = (merged_recs['recStrengthCB'] * cb_weight) + (merged_recs['recStrengthCF'] * cf_weight)
    recommendations = merged_recs.sort_values(by='recStrengthHybrid', ascending=False).head(topn)


    recommendations = recommendations[['content', 'articleId', 'recStrengthHybrid']]
    recommendations = recommendations.reset_index(drop=True)

    print(recommendations)

    return recommendations

# Example usage
user_id = 3
recommendations = hybrid_recommend_items(
    cb_recommender=generate_recommendations_for_user_con,
    cf_recommender=recommend_movies_coll,
    user_id=user_id,
    cb_weight=1.0,
    cf_weight=1.0,
    topn=5
)


Top 1000 Recommendations for User ID 3:
                                              content  articleId  similarity
0   Computer vision is a field that enables machin...          8    0.745435
1   Robots are increasingly becoming part of our d...          4    0.709408
2               Machine learning is a subset of AI...          2    0.684573
3               AI is transforming various sectors...          1    0.609886
4   AI applications in healthcare are revolutioniz...         11    0.592066
5   Data science combines statistics, computer sci...          6    0.588825
6   Deep learning is a technique in machine learni...          3    0.581179
7   Ethical considerations in AI development are c...          5    0.547763
8   Reinforcement learning is a type of machine le...         10    0.528841
9   AI can play a significant role in addressing c...         12    0.507852
10  NLP enables computers to understand human lang...          7    0.464464
11  GANs are a class of machine lear

KeyError: 'recStrengthCB'

In [ ]:
def evaluate_hybrid_recommender(k=10, cb_weight=1.0, cf_weight=1.0, relevance_threshold=3.5):
    precision_sum = 0
    recall_sum = 0
    user_count = 0

    for user_id in user_profiles['userId']:
        user_count += 1

        # Generate hybrid recommendations
        recommendations = hybrid_recommend_items(
            cb_recommender=generate_recommendations_for_user_con,
            cf_recommender=recommend_movies_coll,
            user_id=user_id,
            cb_weight=cb_weight,
            cf_weight=cf_weight,
            topn=k
        )

        if recommendations is None:
            continue

        # Get relevant movies
        relevant_movies = get_relevant_articles(user_id, threshold=relevance_threshold)
        recommended_ids = set(recommendations['articleId'])
        relevant_and_recommended = recommended_ids & relevant_movies

        # Calculate precision and recall
        precision = len(relevant_and_recommended) / k
        recall = len(relevant_and_recommended) / len(relevant_movies) if len(relevant_movies) > 0 else 0

        precision_sum += precision
        recall_sum += recall

    # Calculate average precision, recall, and F1 score
    precision_at_k = precision_sum / user_count
    recall_at_k = recall_sum / user_count
    f1_at_k = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if (precision_at_k + recall_at_k) > 0 else 0

    return precision_at_k, recall_at_k, f1_at_k

# Call the evaluation function for hybrid recommendations
precision_at_k, recall_at_k, f1_at_k = evaluate_hybrid_recommender(k=10)

print(f"Hybrid Precision@{k}: {precision_at_k}")
print(f"Hybrid Recall@{k}: {recall_at_k}")
print(f"Hybrid F1 Score@{k}: {f1_at_k}")


Top 1000 Recommendations for User ID 1:
                                              content  articleId  similarity
0               Machine learning is a subset of AI...          2    0.834161
1               AI is transforming various sectors...          1    0.772954
2   Ethical considerations in AI development are c...          5    0.734210
3   Deep learning is a technique in machine learni...          3    0.707602
4   Robots are increasingly becoming part of our d...          4    0.618873
5   Reinforcement learning is a type of machine le...         10    0.593728
6   AI can play a significant role in addressing c...         12    0.589508
7   AI applications in healthcare are revolutioniz...         11    0.565175
8   Computer vision is a field that enables machin...          8    0.540962
9   GANs are a class of machine learning framework...          9    0.515122
10  NLP enables computers to understand human lang...          7    0.447851
11  Data science combines statistics

KeyError: 'recStrengthCB'

Precision 4 of 0.098 means that, on average, only about 9.8% of the top 4 recommended movies are relevant to the user.

Recall 4 of 0.332 indicates that the system is able to retrieve about 33.2% of all relevant movies for the user within the top 4 recommendations.

F1 Score 4 of 0.151 is relatively low, reflecting a trade-off between precision and recall. It indicates that while the system is somewhat effective in retrieving relevant items, its overall performance is not strong.